In [ ]:
# Clone rag_faq folder from llm4gov repo
!rm -rf llm4gov
!git clone --filter=blob:none --sparse https://github.com/Labic-ICMC-USP/llm4gov.git
%cd llm4gov
!git sparse-checkout set rag_faq
%cd rag_faq
!ls

Cloning into 'llm4gov'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 117 (delta 17), reused 69 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (117/117), 24.65 KiB | 1.45 MiB/s, done.
Resolving deltas: 100% (17/17), done.
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 1 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (1/1), 50 bytes | 50.00 KiB/s, done.
/content/llm4gov
remote: Enumerating objects: 153, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 153 (delta 62), reused 152 (delta 62), pack-reused 0 (from 0)
Receiving objects: 100% (153/153), 13.28 MiB | 17.33 MiB/s, done.
Resolving deltas: 100% (62/62), done.
Updating files: 100% (155/155), done.
/content/llm4gov/rag_faq
config.yaml    load_pdf.ipynb  rag_faq		 run_ind

In [ ]:
# Install requirements
!pip install -e .

Obtaining file:///content/llm4gov/rag_faq
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Running setup.py develop for rag-faq
ERROR: pip's dependency resolver does not currently take into account all the packages that are

## ⚙️ Configuração e carregamento de dados
Define os parâmetros essenciais para gerar FAQs e embeddings.

- **Configuração manual** (ajuste conforme necessário):
  - **csv_path**: Pasta com os arquivos CSV dos PPCs.
  - **course_name**: Nome do curso correspondente ao CSV.
  - **project_name**: Pasta onde serão salvos FAQs e embeddings.
  - **os.environ['OPENROUTER_API_KEY']**: Chave da API do OpenRouter.

In [ ]:
import os, tempfile, yaml
import pandas as pd
from pathlib import Path
from rag_faq.config import load_config
from rag_faq.embedder import embed_faqs
from rag_faq.indexer import generate_faqs

# =============== EDIT THIS MANUALLY =============== #
csv_path = Path("data/ppp_bcc/ppp_bcc_chunks.csv")

course_name = "Bacharelado em Ciência da Computação"

project_name = "test_project"

os.environ['OPENROUTER_API_KEY'] = "API_KEY"
# ================================================== #

if not csv_path.exists():
    raise FileNotFoundError(f"CSV file not found: {csv_path}")

df = pd.read_csv(csv_path)
texts = [str(row.to_dict()) for _, row in df.iterrows()]

api_key = os.getenv("OPENROUTER_API_KEY")
if api_key is None:
    raise ValueError("OpenRouter API key not found in environment variables.")

# Cria arquivo de config temporário
config = load_config("config.yaml")
config["llm"]["faq_generator"]["api_key"] = api_key
config["llm"]["rag_answer"]["api_key"] = api_key
tmp_config = tempfile.NamedTemporaryFile(delete=False, suffix=".yaml")
tmp_config_path = tmp_config.name
tmp_config.close()
with open(tmp_config_path, "w", encoding="utf-8") as f:
    yaml.safe_dump(config, f, allow_unicode=True)

doc_stem = csv_path.parent.name
project_dir = Path(config["paths"]["projects_dir"]) / project_name / doc_stem
project_dir.mkdir(parents=True, exist_ok=True)

print(f"Project folder created at: {project_dir}")

Project folder created at: projects/test_project/ppp_bcc


## Geração de FAQs
Gera perguntas e respostas a partir dos PPCs/CSVs, usando prompts definidos para compor a base de conhecimento do RAG.

- **Saída**:
  - `individual/faq.csv` — FAQ de uma única persona
  - `unificado/faq_aluno.csv`, `unificado/faq_professor.csv`, `unificado/faq_pesquisador.csv` — FAQs por persona
  - `unificado/faq.csv` — FAQ combinado de todas as personas


### 🎯 Geração de FAQs (Persona única)
Gera FAQs para uma única persona (ex.: aluno).

- **Objetivo**:
  - Criar um conjunto de FAQs focado em um tipo específico de usuário
  - Salvar no subdiretório `individual`

- **Saída**: `individual/faq.csv` com perguntas e respostas voltadas à persona.


In [ ]:
# Cria um diretório individual para FAQs de uma única persona
individual_dir = project_dir / "individual"
individual_dir.mkdir(parents=True, exist_ok=True)

persona = "aluno"
generate_faqs(config, individual_dir, texts, course_name, persona)

Generating FAQs: 100%|██████████| 35/35 [07:23<00:00, 12.66s/it]

✅ FAQ saved to: projects/test_project/ppp_bcc/individual/faq.csv


In [ ]:
# Exibe exemplos de dados do faq.csv
individual_df_data = pd.read_csv(f"{individual_dir}/faq.csv")

# Mostrar o cabeçalho do dataframe
display(individual_df_data.sample(5))
print(f"Total de perguntas: {len(individual_df_data)}")

# Exibe um exemplo de pergunta e resposta
if individual_df_data.empty:
    print("DataFrame vazio, sem exemplos para exibir.")
else:
    example_row = individual_df_data.iloc[10]

    print("\nExemplo de Q&A:")
    print(f"Q: {example_row['question']}")
    print(f"A: {example_row['answer']}")

,source_text,question,answer,course,persona
227,"{'doc_name': 'ppp_bcc', 'chunk_id': 23, 'start...",O que é abordado na disciplina SSC0128 – Gerên...,"Conhecimentos, habilidades e técnicas utilizad...",Bacharelado em Ciência da Computação,aluno
61,"{'doc_name': 'ppp_bcc', 'chunk_id': 7, 'start_...",Quais são as atividades extracurriculares ofer...,Programas de Iniciação Científica e Tecnológic...,Bacharelado em Ciência da Computação,aluno
136,"{'doc_name': 'ppp_bcc', 'chunk_id': 14, 'start...",Quais são as disciplinas que desenvolvem a hab...,SSC0904 – Sistemas Computacionais Distribuídos...,Bacharelado em Ciência da Computação,aluno
234,"{'doc_name': 'ppp_bcc', 'chunk_id': 24, 'start...",O que são disciplinas 'básicas'/'essenciais' n...,São todas as disciplinas obrigatórias que todo...,Bacharelado em Ciência da Computação,aluno
334,"{'doc_name': 'ppp_bcc', 'chunk_id': 34, 'start...",Quantas horas de atividades de extensão tem a ...,5 horas.,Bacharelado em Ciência da Computação,aluno


Total de perguntas: 350

Exemplo de Q&A:
Q: Qual é o objetivo principal do curso de Bacharelado em Ciências de Computação do ICMC USP?
A: Preparar um profissional com sólida formação conceitual, teórica e experimental em diferentes áreas de Computação.


### 👥 Geração de FAQs (Multi persona)
Gera FAQs para diferentes personas (aluno, professor, pesquisador).

- **Objetivo**:
  - Criar um conjunto de FAQs focado para cada persona com prompts dedicados
  - Salvar arquivos separados no subdiretório `unificado/`

- **Saídas** (em `unificado/`):
  - `faq_aluno.csv`
  - `faq_professor.csv`
  - `faq_pesquisador.csv`


In [ ]:
# Cria um diretório unificado para FAQs de múltiplas personas
unificado_dir = project_dir / "unificado"
unificado_dir.mkdir(parents=True, exist_ok=True)

persona_type = ["aluno", "professor", "pesquisador"]

for persona in persona_type:
   generate_faqs(config, unificado_dir, texts, course_name, persona, multi_persona=True)

Generating FAQs: 100%|██████████| 35/35 [05:55<00:00, 10.16s/it]


✅ FAQ saved to: projects/test_project/ppp_bcc/unificado/faq_aluno.csv


Generating FAQs: 100%|██████████| 35/35 [07:07<00:00, 12.21s/it]


✅ FAQ saved to: projects/test_project/ppp_bcc/unificado/faq_professor.csv


Generating FAQs: 100%|██████████| 35/35 [10:41<00:00, 18.32s/it]

✅ FAQ saved to: projects/test_project/ppp_bcc/unificado/faq_pesquisador.csv


In [ ]:
# Agrupa FAQs específicas de todas personas em um único arquivo

# Lista de personas para agrupar
persona_files = ["faq_aluno.csv", "faq_professor.csv", "faq_pesquisador.csv"]
all_faqs = []

for file_name in persona_files:
    file_path = unificado_dir / file_name
    if file_path.exists():
        df = pd.read_csv(file_path)
        all_faqs.append(df)
        print(f"✅ Loaded {len(df)} FAQs from {file_name}")
    else:
        print(f"⚠️  File not found: {file_name}")

if all_faqs:
    # Concatena todos os dataframes
    merged_df = pd.concat(all_faqs, ignore_index=True)

    # Salva o arquivo
    merged_path = unificado_dir / "faq.csv"
    merged_df.to_csv(merged_path, index=False, encoding="utf-8")

    print(f"\n📊 Merge Summary:")
    print(f"📈 Total FAQs: {len(merged_df)}")

    # Mostra quantidade de FAQs por persona
    persona_counts = merged_df['persona'].value_counts()
    print(f"👥 FAQs by persona:")
    for persona, count in persona_counts.items():
        print(f"   - {persona}: {count}")

    # Mostra quantidade de FAQs por curso
    print(f"🎓 FAQs by course:")
    course_counts = merged_df['course'].value_counts()
    for course, count in course_counts.items():
        print(f"   - {course}: {count}")

    print(f"\n✅ Combined CSV saved to: {merged_path}")
else:
    print("❌ No FAQ files found to merge!")


✅ Loaded 350 FAQs from faq_aluno.csv
✅ Loaded 350 FAQs from faq_professor.csv
✅ Loaded 350 FAQs from faq_pesquisador.csv

📊 Merge Summary:
📈 Total FAQs: 1050
👥 FAQs by persona:
   - aluno: 350
   - professor: 350
   - pesquisador: 350
🎓 FAQs by course:
   - Bacharelado em Ciência da Computação: 1050

✅ Combined CSV saved to: projects/test_project/ppp_bcc/unificado/faq.csv


In [ ]:
# Exibe exemplos de dados do faq.csv
unificado_df_data = pd.read_csv(f"{unificado_dir}/faq.csv")

# Mostrar o cabeçalho do dataframe
display(unificado_df_data.sample(5))
print(f"Total de perguntas: {len(unificado_df_data)}")

# Exibir um exemplo de pergunta e resposta
if unificado_df_data.empty:
    print("DataFrame vazio, sem exemplos para exibir.")
else:
    example_row = unificado_df_data.iloc[370]

    print("\nExemplo de Q&A:")
    print(f"Q: {example_row['question']}")
    print(f"A: {example_row['answer']}")

,source_text,question,answer,course,persona
179,"{'doc_name': 'ppp_bcc', 'chunk_id': 18, 'start...",Qual é o objetivo do curso em relação ao domín...,Apresentar ao aluno a conceituação teórica das...,Bacharelado em Ciência da Computação,aluno
123,"{'doc_name': 'ppp_bcc', 'chunk_id': 13, 'start...",Quais são os objetivos da disciplina SSC0119 –...,Fornecer ao aluno experiências práticas sobre ...,Bacharelado em Ciência da Computação,aluno
327,"{'doc_name': 'ppp_bcc', 'chunk_id': 33, 'start...",Quantos créditos são exigidos para a conclusão...,211 (147 créditos aula + 64 créditos trabalho).,Bacharelado em Ciência da Computação,aluno
485,"{'doc_name': 'ppp_bcc', 'chunk_id': 14, 'start...",Quais são as habilidades desenvolvidas na disc...,Apresentar os conceitos básicos em redes de co...,Bacharelado em Ciência da Computação,professor
303,"{'doc_name': 'ppp_bcc', 'chunk_id': 31, 'start...",Quais são as disciplinas do segundo semestre l...,"No segundo semestre letivo, as disciplinas são...",Bacharelado em Ciência da Computação,aluno


Total de perguntas: 1050

Exemplo de Q&A:
Q: Quais são as três áreas de atuação definidas para o egresso do curso de Bacharelado em Ciências de Computação?
A: As três áreas de atuação definidas são: 1. Mercado de trabalho, que engloba indústrias de computadores, empresas de software e setores de processamento de dados de instituições públicas e privadas; 2. Atividades de pesquisa; 3. Ações de empreendedorismo na área de computação.


## Geração de Embeddings
Cria representações vetoriais (embeddings) para todas as perguntas das FAQs, permitindo buscas por similaridade.

- **Saída**: `embeddings.npy` — arquivo com as representações vetoriais das perguntas.


In [ ]:
# Gera embeddings somente se as respectivas etapas de geração de FAQ foram executadas
if unificado_dir:
    embed_faqs(config, unificado_dir)

if individual_dir:
    embed_faqs(config, individual_dir)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Generating embeddings: 100%|██████████| 1050/1050 [00:56<00:00, 18.73it/s]


✅ Embeddings saved to: projects/test_project/ppp_bcc/unificado


Generating embeddings: 100%|██████████| 350/350 [00:16<00:00, 21.12it/s]

✅ Embeddings saved to: projects/test_project/ppp_bcc/individual


## Testes

### Teste 1 — FAQs e embeddings gerados neste notebook

Executa um teste rápido usando a base criada acima:
- Consulta na base individual (persona única: aluno)
- Consulta na base unificada (multi‑persona: aluno, professor, pesquisador)

In [ ]:
# Consulta direta via funções (individual e unificado)

from rag_faq.generator import generate_rag_answer

question = "Qual é a carga horária do curso?"
print(f"Question: \n{question}\n")

print("Resposta individual (persona única: aluno):")
result_individual = generate_rag_answer(config, individual_dir, question)
print(result_individual["answer"])

print("\nResposta unificado (multi-persona: aluno, professor, pesquisador):")
result_unificado = generate_rag_answer(config, unificado_dir, question)
print(result_unificado["answer"])


Question: 
Qual é a carga horária do curso?

Resposta individual (persona única: aluno):
Resposta Final: O curso tem uma carga horária total de 4575 horas.

Evidência de Apoio: 
- O documento [1] não fornece explicitamente a carga horária, mas menciona várias exigências do curso.
- O documento [2] e [3] afirmam explicitamente que "No geral, o aluno deve cursar 241 créditos, que totalizam 4575 horas de estudo".
- O documento [4] e [5] também confirmam que o total de créditos exigidos é 241, composto por 211 créditos obrigatórios e 30 créditos optativos, totalizando 4575 horas de estudo.

Resposta unificado (multi-persona: aluno, professor, pesquisador):
Resposta Final: O curso de Bacharelado em Ciências de Computação exige um total de 4575 horas de estudo, distribuídos em 241 créditos, que são compostos por:

- 211 créditos obrigatórios (147 créditos aula e 64 créditos trabalho)
- 30 créditos aula em disciplinas optativas

Além disso, os alunos devem cumprir:
- 90 horas de Atividades Ac

In [ ]:
# Consulta via CLI (base individual)
!python -m rag_faq.main --mode query --project {project_name}/{doc_stem}/individual --config {tmp_config_path}

2025-10-22 14:58:38.584461: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761145118.623825    8642 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761145118.635819    8642 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761145118.672344    8642 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761145118.672417    8642 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761145118.672422    8642 computation_placer.cc:177] computation placer alr

In [ ]:
# Consulta via CLI (base unificada)
!python -m rag_faq.main --mode query --project {project_name}/{doc_stem}/unificado --config {tmp_config_path}


2025-10-22 14:59:14.532490: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761145154.574206    8810 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761145154.587530    8810 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761145154.618057    8810 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761145154.618115    8810 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761145154.618126    8810 computation_placer.cc:177] computation placer alr

### Teste 2 — FAQs e embeddings pré‑gerados

Executa um teste usando bases já existentes, combinando embeddings de todos os cursos do ICMC.
  - Consulta na base individual (persona única)
  - Consulta na base unificada (multi‑persona)

In [ ]:
# Consulta direta via funções (individual e unificado)

from rag_faq.generator import generate_rag_answer

project_name = "myproj"
doc_stem = "ppp_all_courses"
project_dir = os.path.join(config["paths"]["projects_dir"], f"{project_name}/{doc_stem}")

all_courses_individual_dir = project_dir + "/individual"
all_courses_unificado_dir = project_dir + "/unificado"

question = "Qual é a diferença do curso de Engenharia da computação e Ciência da computação?"
print(f"Question: \n{question}\n")

print("Resposta individual (persona única: aluno):")
result_individual = generate_rag_answer(config, all_courses_individual_dir, question)
print(result_individual["answer"])

print("\nResposta unificado (multi-persona: aluno, professor, pesquisador):")
result_unificado = generate_rag_answer(config, all_courses_unificado_dir, question)
print(result_unificado["answer"])

Question: 
Qual é a diferença do curso de Engenharia da computação e Ciência da computação ?

Resposta individual (persona única: aluno):
Os documentos fornecidos não explicitam as diferenças entre os cursos de Engenharia da Computação e Ciência da Computação. No entanto, podemos inferir algumas distinções com base nas informações disponíveis.

O curso de Engenharia de Computação parece ter uma abordagem mais prática e tecnológica, com ênfase em áreas como engenharia eletrônica, computação e áreas de interface. O perfil do egresso de Engenharia de Computação é descrito como tendo uma sólida formação técnico-científica e profissional geral, capacitando-o a absorver e desenvolver novas tecnologias.

Já o curso de Ciência da Computação, mencionado no documento [1], parece ter uma abordagem mais teórica e focada em conceitos de computação, com disciplinas como Introdução à Ciência de Computação, Algoritmos e Estruturas de Dados, e Programação Orientada a Objetos.

Embora não seja possível 

In [ ]:
# Consulta via CLI (base individual)
!python -m rag_faq.main --mode query --project {project_name}/{doc_stem}/individual --config {tmp_config_path}

2025-10-22 15:00:07.798313: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761145207.823737    9034 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761145207.832975    9034 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761145207.856262    9034 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761145207.856305    9034 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761145207.856312    9034 computation_placer.cc:177] computation placer alr

In [ ]:
# Consulta via CLI (base unificada)
!python -m rag_faq.main --mode query --project {project_name}/{doc_stem}/unificado --config {tmp_config_path}

2025-10-22 15:01:21.803772: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761145281.828359    9346 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761145281.836765    9346 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761145281.856494    9346 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761145281.856539    9346 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761145281.856544    9346 computation_placer.cc:177] computation placer alr